# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [19]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

In [5]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Ebac/Profissão: Cientista de dados/Ciência de dados/Módulo 13/Ex 1/previsao_de_renda.csv')

Mounted at /content/drive


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [7]:
# Separe a base de dados df em treinamento e teste (25% para teste, 75% para treinamento).

X = df.drop('renda', axis=1)
y = df['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [8]:
# Regularização usando o Ridge (l1_wt muito pequeno)
alfa = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
for n in alfa:
  modelo = 'np.log(renda) ~ C(sexo, Treatment("F")) + posse_de_veiculo + posse_de_imovel + qtd_filhos + idade + tempo_emprego'
  md = smf.ols(modelo, data = df)
  reg = md.fit_regularized(method = 'elastic_net'
                          , refit = True
                          , L1_wt = 0.0000001
                          , alpha = n)
  print("<^>"*30)
  print(f"\n Alpha: {n}")
  print(reg.summary())

<^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^>

 Alpha: 0
                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     940.9
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        02:09:57   Log-Likelihood:                -13673.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12420   BIC:                         2.742e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
-----------

Quando olhamos os modelos regularizados utilizando o método _ridge_, temos os valores de $R^2$ iguais para todos os alphas

In [9]:
# Regularização usando o Lasso
alfa = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
for n in alfa:
  modelo = 'np.log(renda) ~ C(sexo, Treatment("F")) + posse_de_veiculo + posse_de_imovel + qtd_filhos + idade + tempo_emprego'
  md = smf.ols(modelo, data = df)
  reg = md.fit_regularized(method = 'elastic_net'
                          , refit = True
                          , L1_wt = 1
                          , alpha = n)
  print("<^>"*30)
  print(f"\n Alpha: {n}")
  print(reg.summary())

<^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^><^>

 Alpha: 0
                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     940.9
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        02:09:58   Log-Likelihood:                -13673.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12420   BIC:                         2.742e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
-----------

Utilizando o método Lasso, temos então modelos com $R^2$ diferentes, onde o valor do alpha = 0 possui melhor valor de $R^2$ e $R^2_{ajustado}$

In [10]:
# Alterando os valores das colunas abaixo para valores inteiros
df['posse_de_veiculo'] = [int(x) for x in df['posse_de_veiculo']]
df['posse_de_imovel'] = [int(x) for x in df['posse_de_imovel']]

In [11]:
# Utilizando apenas dados numéricos
apenas_num = df.select_dtypes(include=['int64', 'float64'])
apenas_num = apenas_num.dropna(axis=0)

# Separando os dados de treino e teste
X = apenas_num.drop('renda', axis=1)
y = apenas_num['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.05,
                       threshold_out = 0.05,
                       verbose=True):
    """ Perform a forward-backward feature selection
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X_train, y_train)

print(variaveis)

Add  tempo_emprego                  with p-value 0.0
#############
['tempo_emprego']
Add  posse_de_veiculo               with p-value 4.09115e-24
#############
['tempo_emprego', 'posse_de_veiculo']
Add  qt_pessoas_residencia          with p-value 0.00362065
#############
['tempo_emprego', 'posse_de_veiculo', 'qt_pessoas_residencia']
Add  qtd_filhos                     with p-value 0.0374331
#############
['tempo_emprego', 'posse_de_veiculo', 'qt_pessoas_residencia', 'qtd_filhos']
#############
['tempo_emprego', 'posse_de_veiculo', 'qt_pessoas_residencia', 'qtd_filhos']
['tempo_emprego', 'posse_de_veiculo', 'qt_pessoas_residencia', 'qtd_filhos']


In [16]:
# Rodando o modelo
modelo = 'renda ~ tempo_emprego + posse_de_veiculo + qt_pessoas_residencia + qtd_filhos'
md = smf.ols(modelo, data = df)
md.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.162
Method:                 Least Squares   F-statistic:                     601.8
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        02:10:45   Log-Likelihood:            -1.2953e+05
No. Observations:               12427   AIC:                         2.591e+05
Df Residuals:                   12422   BIC:                         2.591e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              -119.8575    341.445     -0.351      0.726    -789.143     549.428
tempo_emprego           523.2390     10.928     47.880      0.000     501.818     544.660
posse_de_veiculo       1933.2717    149.831     12.903      0.000    1639.579    2226.964
qt_pessoas_residencia   726.3882    181.708      3.998      0.000     370.213    1082.563
qtd_filhos             -578.4275    215.510     -2.684      0.007   -1000.860    -155.995
==============================================================================
Omnibus:                    17092.864   Durbin-Watson:                   2.035
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          7878465.995
Skew:                           7.767   Prob(JB):                         0.00
Kurtosis:                     125.369   Cond. No.                         60.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Podemos observar que o $R^2$ do modelo acima é mais baixo que o dos modelos anteriores, onde o melhor modelo gerado até foi o utilizando o método Lasso, que obteve um r_score mais e r_score_ajustado maior

Criando uma árvore de decisão

In [26]:
# Dividir os dados em conjuntos de treinamento e teste
X = apenas_num.drop('renda', axis=1)
y = apenas_num['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Criar o modelo de árvore de decisão
modelo_arvore = DecisionTreeRegressor()

# Treinar o modelo com os dados de treinamento
modelo_arvore.fit(X_train, y_train)

# Fazer previsões nos dados de teste
previsoes = modelo_arvore.predict(X_test)

# Avaliar o desempenho do modelo
mse = mean_squared_error(y_test, previsoes)
r2 = r2_score(y_test, previsoes)
print(f'Mean Squared Error: {mse} \nR2 score: {r2}')


Mean Squared Error: 58960266.23287246 
R2 score: -0.041228710805783164


Como podemos ver, utilizando o método utilizando uma árvore de decisão gerou um resultado muito ruim, trazendo um modelo que não possui previsibilidade